In [10]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import label_ranking_average_precision_score, top_k_accuracy_score
from torchvision.models import inception_v3

# Paths to the train, test, and validation data folders
train_dir = 'split_data/train'
test_dir = 'split_data/test'
val_dir = 'split_data/val'

# Image dimensions
img_width, img_height = 299, 299

# Batch size
batch_size = 32

# Number of input channels
input_channels = 3

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Data loaders
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Load the InceptionV3 model
base_model = inception_v3(pretrained=True)

# Modify the input layer to accept the specified number of input channels
base_model.Conv2d_1a_3x3.conv = nn.Conv2d(input_channels, 32, kernel_size=3, stride=2, bias=False)

# Remove the original classifier
num_features = base_model.fc.in_features
base_model.fc = nn.Identity()

# Create a new classifier
classifier = nn.Linear(num_features, len(train_dataset.classes))

# Combine the base model and the classifier
model = nn.Sequential(base_model, classifier)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        # Get the output of the base model
        outputs, _ = model[0](inputs)
        outputs = model[1](outputs)  # Pass the base model output through the linear layer
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, epochs, epoch_loss, epoch_acc * 100))
    
# Evaluation
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    # Get the output of the base model
    outputs = model[0](inputs)
    outputs = model[1](outputs)  # Pass the base model output through the linear layer
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


# Convert predictions to one-hot encoded format
y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
y_pred_onehot = torch.zeros(y_pred.size(0), len(test_dataset.classes))
y_pred_onehot.scatter_(1, y_pred.view(-1, 1), 1)

# Calculate accuracy and loss
accuracy = (y_true == y_pred).sum().item() / len(y_true)
loss = criterion(outputs, labels).item()

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('Loss:', loss)

# Calculate LWLRAP score
lwlrAP = label_ranking_average_precision_score(y_true.cpu().numpy(), y_pred_onehot.numpy())
print('LWLRAP score:', lwlrAP)

# Calculate top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(y_true.cpu().numpy(), y_pred_onehot.numpy(), k=5)
print('Top-5 categorical accuracy:', top5_accuracy)

Epoch [1/10], Loss: 1.2157, Accuracy: 62.73%
Epoch [2/10], Loss: 0.7694, Accuracy: 76.63%
Epoch [3/10], Loss: 0.6109, Accuracy: 81.12%
Epoch [4/10], Loss: 0.5222, Accuracy: 83.96%
Epoch [5/10], Loss: 0.4366, Accuracy: 86.81%
Epoch [6/10], Loss: 0.3774, Accuracy: 88.10%
Epoch [7/10], Loss: 0.3148, Accuracy: 89.96%
Epoch [8/10], Loss: 0.3127, Accuracy: 90.59%
Epoch [9/10], Loss: 0.2336, Accuracy: 92.86%
Epoch [10/10], Loss: 0.2127, Accuracy: 93.33%


ValueError: too many values to unpack (expected 2)

In [24]:
import torch.nn.functional as F

# Convert predictions to one-hot encoded format
y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
y_pred_onehot = torch.zeros(y_pred.size(0), len(test_dataset.classes))
y_pred_onehot.scatter_(1, y_pred.view(-1, 1), 1)

# Convert y_true to one-hot encoded format and then numpy array
y_true_onehot = F.one_hot(y_true.squeeze(), num_classes=len(test_dataset.classes)).numpy()

# Calculate accuracy and loss
accuracy = (y_true == y_pred).sum().item() / len(y_true)
loss = criterion(outputs, labels).item()

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('Loss:', loss)

# Calculate LWLRAP score
lwlrAP = label_ranking_average_precision_score(y_true_onehot, y_pred_onehot.numpy())
print('LWLRAP score:', lwlrAP)

# Calculate top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(y_true, y_pred_onehot.numpy(), k=5)
print('Top-5 categorical accuracy:', top5_accuracy)


Accuracy: 79.60%
Loss: 0.038842275738716125
LWLRAP score: 0.8096317280453257
Top-5 categorical accuracy: 0.8441926345609065


C:\Users\arham\AppData\Local\Temp\ipykernel_33236\3637968709.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.tensor(y_true)
C:\Users\arham\AppData\Local\Temp\ipykernel_33236\3637968709.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)
